# Librerias Necesarias

In [1]:
#!pip install umap
#!pip install mlxtend
#!!pip install 'pami[gpu]'

In [20]:
import gc
import matplotlib.pyplot as plt
import mlxtend.frequent_patterns as FP
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import shap
from statsmodels.stats.weightstats import DescrStatsW
import warnings

In [21]:
import sys
sys.setrecursionlimit(16385)

In [22]:
#Libreamos espacio en la GPU
gc.collect()

35465

In [23]:
#Para visualizar todas las columnas
pd.set_option('display.max_columns', None)

In [24]:
#Eliminar las advertencias 
warnings.filterwarnings("ignore")

# Lectura de Datos 
Cargamos los datasets resultantes

In [25]:
base_path = "./ConjuntosDatos/"
endutih_path = base_path + "ENDUTIH2020/conjuntos_de_datos/"

In [26]:
endutih_vivhogar = pd.read_csv(endutih_path+"viv_hog_agrupados.csv",  dtype={"ESTRATO":str, "Grupo":str})
endutih_res = pd.read_csv(endutih_path+"residente_agrupados.csv",  dtype={"ESTRATO":str, "Grupo":str})
endutih_usr = pd.read_csv(endutih_path+"usuario_agrupados.csv", low_memory=False,  dtype={"ESTRATO":str, "Grupo":str})
endutih_usr2 = pd.read_csv(endutih_path+"usuario2_agrupados.csv", low_memory=False,  dtype={"ESTRATO":str, "Grupo":str})
estados = pd.read_csv(base_path+"CodigoEdos.csv")
columnas_sociotec = pd.read_csv("./columnas_clustering/cols_clus_medio_bajo.csv").columns

endutih_completa = pd.merge(endutih_vivhogar,endutih_res.drop(["UPM_DIS","ESTRATO","ENT","DOMINIO", "EST_DIS", "TLOC","Grupo"], axis=1), on = ["UPM","VIV_SEL", "HOGAR"])
endutih_completa = pd.merge(endutih_completa, estados["Codigo ENT".split()], left_on=["ENT"], right_on="Codigo", suffixes=("_orig","")).drop("Codigo ENT_orig".split(), axis=1)

In [27]:
endutih_completa["llave_tmp"] = endutih_completa["ESTRATO"]+endutih_completa["DOMINIO"]+endutih_completa["Grupo"]+endutih_completa["ENT"]

In [28]:
endutih_completa.head()

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT,llave_tmp
0,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,1,1,1,1,2,54,27.0,3.0,2.0,0.0,0.0,9.0,9.0,1.0,6.0,4.0,NaN,575,B.C.,1U11B.C.
1,2,1,1,1,1,2,2,2,1,1.0,572,2,1,1,2,2,1,2.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,575,U,1,1,11,206264,570,17,1,1,2,3,2,22,7.0,1.0,2.0,2.0,3.0,2.0,2.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.
2,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,1,1,2,36,29.0,6.0,2.0,3.0,3.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.
3,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,2,3,1,20,29.0,3.0,2.0,6.0,2.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.
4,2,1,1,1,1,1,1,6,1,1.0,572,2,1,2,1,1,1,5.0,5.0,2,1,1,0.0,1,1.0,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1,1.0,2.0,2.0,2.0,0.0,2,1,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,1.0,400.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,1.0,3.0,575,U,1,1,11,206264,570,17,1,2,3,3,2,19,20.0,2.0,2.0,6.0,3.0,1.0,1.0,1.0,1.0,NaN,1.0,575,B.C.,1U11B.C.


In [29]:
resumen_idtmex = pd.read_csv(base_path+"long_IDTMex_desglosado_dominio_grupos_edo.csv", index_col=0, dtype={"ESTRATO":str, "Grupo":str})
resumen_idtmex

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
0,2,R,20,Ags.,0.526023,0.418287,0.463184,4.703610,349697
1,2,R,21,Ags.,1.000000,0.493244,0.506113,6.985203,1949
2,2,R,22,Ags.,0.639535,0.606965,0.521212,6.028424,1474
3,2,R,23,Ags.,0.500000,0.666667,0.333333,5.333333,544
4,2,U,20,Ags.,0.534371,0.415015,0.463455,4.724455,450683
...,...,...,...,...,...,...,...,...,...
515,3,U,32,Zac.,0.750000,0.500000,0.500000,6.000000,298
516,3,U,33,Zac.,0.719707,0.623086,0.640362,6.651896,274852
517,3,U,34,Zac.,0.847005,0.586430,0.666667,7.067073,619
518,4,U,42,Zac.,0.805036,0.756772,0.789014,7.825261,94796


In [30]:
resumen_idtmex.describe(include="all")

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
count,520,520,520,520,520.000000,520.000000,520.000000,520.000000,5.200000e+02
unique,4,2,15,32,NaN,NaN,NaN,NaN,NaN
top,2,U,20,B.C.S,NaN,NaN,NaN,NaN,NaN
freq,192,418,62,20,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,0.683853,0.565991,0.618888,6.237152,2.438776e+05
std,NaN,NaN,NaN,NaN,0.212327,0.207891,0.191084,1.849581,6.543174e+05
min,NaN,NaN,NaN,NaN,0.123636,0.000000,0.166667,1.511805,1.020000e+02
25%,NaN,NaN,NaN,NaN,0.510543,0.445805,0.463121,4.934255,3.036500e+03
50%,NaN,NaN,NaN,NaN,0.701144,0.598388,0.596687,6.567132,9.642000e+03
75%,NaN,NaN,NaN,NaN,0.856716,0.668162,0.777778,7.666667,1.648105e+05


# Estratificación de resultados

In [31]:
np.linspace(1.51,10,5)

array([ 1.51  ,  3.6325,  5.755 ,  7.8775, 10.    ])

In [32]:
condiciones = [(resumen_idtmex.IDTMex<=3.63), (resumen_idtmex.IDTMex<=5.76), (resumen_idtmex.IDTMex<=7.88), (resumen_idtmex.IDTMex>7.88)]
etiquetas = "Bajo Medio-bajo Medio-alto Alto".split()

resumen_idtmex["clasificacion"] = np.select(condiciones, etiquetas)
resumen_idtmex.head()

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR,clasificacion
0,2,R,20,Ags.,0.526023,0.418287,0.463184,4.703610,349697,Medio-bajo
1,2,R,21,Ags.,1.000000,0.493244,0.506113,6.985203,1949,Medio-alto
2,2,R,22,Ags.,0.639535,0.606965,0.521212,6.028424,1474,Medio-alto
3,2,R,23,Ags.,0.500000,0.666667,0.333333,5.333333,544,Medio-bajo
4,2,U,20,Ags.,0.534371,0.415015,0.463455,4.724455,450683,Medio-bajo


In [33]:
resumen_grupos = resumen_idtmex.groupby("clasificacion")["FAC_HOGAR IDTMex".split()].agg(
    prom=("IDTMex",lambda x: np.average(x, weights=resumen_idtmex.loc[x.index, "FAC_HOGAR"])),
    var=("IDTMex",lambda x:  DescrStatsW(x, weights=resumen_idtmex.loc[x.index, "FAC_HOGAR"], ddof=0).var),
    tot=( "FAC_HOGAR","sum")
)
resumen_grupos["perc"] = 100*resumen_grupos['tot']/resumen_grupos["tot"].sum()
resumen_grupos.loc[etiquetas]

,prom,var,tot,perc
clasificacion,,,,
Bajo,2.844245,0.198836,25241921,19.904315
Medio-bajo,4.954035,0.302429,55754317,43.964621
Medio-alto,6.771159,0.354268,41115193,32.421056
Alto,8.347274,0.116710,4704896,3.710008


In [34]:
x = resumen_idtmex.IDTMex
prom = np.average(x, weights=resumen_idtmex["FAC_HOGAR"])
var = DescrStatsW(x, weights=resumen_idtmex["FAC_HOGAR"], ddof=0).var
prom, var

(5.249115972689082, 2.5883331036784245)

# Seleccion de grupos a analizar
Se sleccionan los grupos que se encuentran en el límite para subir la clasificación

In [35]:
grupos_bajos = resumen_idtmex[resumen_idtmex.clasificacion=="Bajo"].sort_values("IDTMex", ascending=False)
grupos_bajos["llave_tmp"] = grupos_bajos["ESTRATO"]+grupos_bajos["DOMINIO"]+grupos_bajos["Grupo"]+grupos_bajos["ENT"]
grupos_bajos = grupos_bajos[grupos_bajos["IDTMex"] >= grupos_bajos["IDTMex"].quantile(0.9)]
grupos_bajos

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR,clasificacion,llave_tmp
163,1,R,11,Gto.,0.403872,0.319342,0.364866,3.622589,922754,Bajo,1R11Gto.
165,2,R,20,Gto.,0.385818,0.295996,0.400745,3.528747,996157,Bajo,2R20Gto.
18,1,R,11,B.C.,0.370303,0.270911,0.423337,3.411529,187094,Bajo,1R11B.C.
355,2,R,20,Q. Roo.,0.352222,0.299472,0.398243,3.403262,57498,Bajo,2R20Q. Roo.
488,1,U,11,Yuc.,0.368952,0.278938,0.391396,3.374351,495403,Bajo,1U11Yuc.
333,1,R,11,Qro.,0.377838,0.279374,0.372411,3.373671,955165,Bajo,1R11Qro.


In [36]:
grupos_bajos.describe()

,acceso,uso,aptitudes,IDTMex,FAC_HOGAR
count,6.000000,6.000000,6.000000,6.000000,6.000000
mean,0.376501,0.290672,0.391833,3.452358,602345.166667
std,0.017451,0.017797,0.021060,0.101161,415441.935485
min,0.352222,0.270911,0.364866,3.373671,57498.000000
25%,0.369290,0.279047,0.377157,3.381579,264171.250000
50%,0.374071,0.287685,0.394820,3.407395,709078.500000
75%,0.383823,0.298603,0.400120,3.499442,947062.250000
max,0.403872,0.319342,0.423337,3.622589,996157.000000


In [48]:
DescrStatsW(grupos_bajos["acceso uso aptitudes IDTMex".split()], weights=grupos_bajos["FAC_HOGAR"], ddof=0).mean

array([0.38466919, 0.29398202, 0.38394421, 3.48249323])

In [49]:
DescrStatsW(grupos_bajos["FAC_HOGAR".split()], weights=grupos_bajos["FAC_HOGAR"], ddof=0).mean

array([841122.99056632])

In [50]:
casos_bajos = endutih_completa[endutih_completa["llave_tmp"].isin(grupos_bajos["llave_tmp"])]
casos_bajos = pd.merge(casos_bajos, estados["Codigo ENT".split()], on = "ENT") 
casos_bajos

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT,llave_tmp,Codigo
0,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,1,1,1,42,28.0,7.0,2.0,2.0,6.0,2.0,2.0,1.0,1.0,NaN,3.0,699,B.C.,1R11B.C.,2
1,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,2,2,2,37,9.0,12.0,2.0,2.0,6.0,2.0,1.0,1.0,6.0,1.0,4.0,699,B.C.,1R11B.C.,2
2,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,3,3,2,15,18.0,2.0,1.0,3.0,3.0,2.0,1.0,1.0,5.0,4.0,NaN,699,B.C.,1R11B.C.,2
3,2,2,2,1,2,2,2,4,1,1.0,699,1,1,2,2,2,1,3.0,2.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,3.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,1,4,3,1,7,17.0,11.0,1.0,2.0,2.0,2.0,1.0,1.0,NaN,NaN,NaN,699,B.C.,1R11B.C.,2
4,2,1,2,1,1,1,1,5,1,1.0,699,2,2,2,1,2,1,4.0,4.0,2,2,2,1.0,1,1.0,1.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,1.0,2,1,2,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,0.0,1.0,389.0,2.0,3.0,3.0,699,R,4,1,11,260010,631,15,1,2,1,1,1,32,7.0,11.0,2.0,6.0,3.0,2.0,1.0,1.0,1.0,NaN,1.0,699,B.C.,1R11B.C.,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4909,2,1,2,1,1,1,2,3,1,1.0,306,2,1,2,2,2,2,0.0,0.0,2,2,2,1.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.0,0.0,0.0,0.0,1.0,2,2,2,2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,306,U,3,1,11,3104022,8038,321,1,17,3,5,2,15,7.0,8.0,2.0,3.0,3.0,2.0,2.0,2.0,6.0,4.0,NaN,306,Yuc.,1U11Yuc.,31
4910,3,1,2,1,1,1,2,4,1,1.0,306,2,2,2,1,2,1,2.0,2.0,2,2,1,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,2.0,1.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,209.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,306,U,3,1,11,3104022,8038,321,1,20,1,1,1,45,5.0,5.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,NaN,2.0,306,Yuc.,1U11Yuc.,31
4911,3,1,2,1,1,1,2,4,1,1.0,306,2,2,2,1,2,1,2.0,2.0,2,2,1,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,2.0,1.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,209.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,306,U,3,1,11,3104022,8038,321,1,20,2,2,2,39,24.0,12.0,2.0,3.0,3.0,2.0,2.0,1.0,1.0,NaN,1.0,306,Yuc.,1U11Yuc.,31
4912,3,1,2,1,1,1,2,4,1,1.0,306,2,2,2,1,2,1,2.0,2.0,2,2,1,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,2.0,1.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,209.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,306,U,3,1,11,3104022,8038,321,1,20,3,3,1,10

In [51]:
casos_bajos["DOMINIO Grupo Codigo".split()].drop_duplicates().to_csv("./ConjuntosDatos/casos_limite.csv", index=False)

# Estados con un valor alto

In [54]:
edos_mejores = resumen_idtmex[resumen_idtmex["ENT"].isin("C.D.Mx|N.L|Edo.Méx.|B.C.|Son.|Jal.".split("|"))]
edos_mejores

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR,clasificacion
18,1,R,11,B.C.,0.370303,0.270911,0.423337,3.411529,187094,Bajo
19,1,U,11,B.C.,0.600000,0.446377,0.430409,5.046326,15525,Medio-bajo
20,2,R,20,B.C.,0.512853,0.447157,0.431790,4.703621,208506,Medio-bajo
21,2,R,22,B.C.,0.750000,0.888889,0.500000,7.555556,3115,Medio-alto
22,2,U,20,B.C.,0.644876,0.618931,0.492148,6.039523,1606767,Medio-alto
...,...,...,...,...,...,...,...,...,...,...
416,4,U,40,Son.,1.000000,1.000000,1.000000,10.000000,2980,Alto
417,4,U,41,Son.,0.825556,0.771358,0.948582,8.284819,3124,Alto
418,4,U,42,Son.,0.823809,0.936457,0.813874,8.668811,368376,Alto
419,4,U,43,Son.,1.000000,1.000000,1.000000,10.000000,1518,Alto


In [56]:
edos_mejores[edos_mejores["Grupo"]=="23"]

,ESTRATO,DOMINIO,Grupo,ENT,acceso,uso,aptitudes,IDTMex,FAC_HOGAR,clasificacion
25,2,U,23,B.C.,0.582504,0.467043,0.286306,4.770804,5617,Medio-bajo
138,2,U,23,C.D.Mx,0.500000,0.333333,0.666667,4.666667,1816,Medio-bajo
208,2,R,23,Jal.,0.500000,0.666667,0.500000,5.666667,5400,Medio-bajo
212,2,U,23,Jal.,0.610495,0.592235,0.395756,5.602432,14278,Medio-bajo
410,2,U,23,Son.,1.000000,0.833333,0.500000,8.333333,824,Alto


In [59]:
endutih_completa[endutih_completa["llave_tmp"] == "2U23C.D.Mx"]

,P1_1,P1_2,P1_3,P1_4,P1_5_1,P1_5_2,P1_5_3,P2_1,P2_2,P2_3,FAC_VIV,P4_1_1,P4_1_2,P4_1_3,P4_1_4,P4_1_5,P4_1_6,P4_1_6_1,P4_1_6_2,P4_2_1,P4_2_2,P4_2_3,P4_3,P4_4,P4_5,P4_6_1,P4_6_2,P4_6_3,P4_6_4,P4_6_5,P4_6_6,P4_7_1,P4_7_2,P4_7_3,P4_7_4,P4_8,P5_1,P5_2_1,P5_2_2,P5_3_1,P5_3_2,P5_4,P5_5,P5_6_1,P5_6_2,P5_6_3,P5_6_4,P5_6_5,P5_7_1,P5_7_2,P5_8_2,P5_9_2,P5_10_2,P5_7_3,P5_8_3,P5_7_4,P5_8_4,P5_9_4,P5_7_5,P5_8_5,P5_7_6,P5_8_6,P5_7_7,P5_8_7,P5_7_8,P5_8_8,P5_11_1,P5_11_2,P5_11_3,FAC_HOG,DOMINIO,TLOC,ESTRATO,Grupo,UPM,UPM_DIS,EST_DIS,HOGAR,VIV_SEL,NUM_REN,PAREN,SEXO,EDAD,DIA,MES,P3_7,NIVEL,GRADO,P3_9_1,P3_9_2,P3_9_3,P3_10,P3_11,P3_12,FAC_HOGAR,ENT,llave_tmp
51477,2,2,1,1,1,1,1,2,1,1.0,905,1,2,2,1,2,1,1.0,1.0,2,1,2,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1,1.0,2.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,9999.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,908,U,1,2,23,916726,2499,80,1,1,1,1,1,46,13.0,2.0,2.0,8.0,5.0,1.0,1.0,1.0,1.0,NaN,4.0,908,C.D.Mx,2U23C.D.Mx
51478,2,2,1,1,1,1,1,2,1,1.0,905,1,2,2,1,2,1,1.0,1.0,2,1,2,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1,1.0,2.0,2.0,2.0,0.0,2,2,1,2,1,2,2.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,9999.0,2.0,0.0,2.0,0.0,3.0,2.0,3.0,908,U,1,2,23,916726,2499,80,1,1,2,5,2,76,24.0,7.0,2.0,3.0,3.0,1.0,1.0,1.0,6.0,4.0,NaN,908,C.D.Mx,2U23C.D.Mx


In [32]:
for ent in edos_mejores["ENT"].unique():
    print(ent)
    print(edos_mejores[edos_mejores["ENT"]==ent].describe())
    print(edos_mejores.loc[edos_mejores["ENT"]==ent,"clasificacion"].value_counts())
    print("-"*20)

B.C.
          acceso        uso  aptitudes     IDTMex     FAC_HOGAR
count  18.000000  18.000000  18.000000  18.000000  1.800000e+01
mean    0.759033   0.708130   0.615872   7.100396  7.040302e+06
std     0.182459   0.185973   0.176297   1.675368  1.345526e+07
min     0.370303   0.270911   0.286306   3.411529  4.622600e+04
25%     0.611219   0.642323   0.490639   6.124496  2.302370e+05
50%     0.757622   0.777197   0.600629   7.791346  3.695385e+05
75%     0.914762   0.813413   0.729189   8.028125  9.273448e+06
max     1.000000   1.000000   0.900093   9.466667  5.211225e+07
Alto          8
Medio-alto    6
Medio-bajo    3
Bajo          1
Name: clasificacion, dtype: int64
--------------------
C.D.Mx
          acceso        uso  aptitudes     IDTMex     FAC_HOGAR
count  16.000000  16.000000  16.000000  16.000000  1.600000e+01
mean    0.758381   0.751932   0.696387   7.434027  6.594524e+06
std     0.164228   0.169520   0.120339   1.401423  1.372612e+07
min     0.500000   0.333333   0.45098

In [36]:
estados[estados["ENT"].isin("C.D.Mx|N.L|Edo.Méx.|B.C.|Son.|Jal.".split("|"))]

,Codigo,Nombre,ENT
1,2,Baja California,B.C.
8,9,Ciudad de México,C.D.Mx
13,14,Jalisco,Jal.
25,26,Sonora,Son.
